# 形態素解析
## 夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．なお，問題37, 38, 39はmatplotlibもしくはGnuplotを用いるとよい．

#### https://nlp100.github.io/ja/ch04.html

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/iamtatsuki05/NLP_100/blob/fix_all_merge/NLP_100_4.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!wget https://nlp100.github.io/data/neko.txt

In [ ]:
!apt install mecab libmecab-dev mecab-ipadic-utf8

In [ ]:
!mecab -o ./neko.txt.mecab ./neko.txt

In [ ]:
!head -80 ./neko.txt.mecab

In [ ]:
# 参考https://qiita.com/Ninagawa_Izumi/items/c90cccb453e2a6fc4466
# http://www.mwsoft.jp/programming/munou/mecab_command.html

#  形態素解析結果の読み込み
## 形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

In [ ]:
text_file_path = './neko.txt.mecab'
samples = []
tokens = []   #初期化

with open(text_file_path, mode='r') as f:
    for line in f:
        if line != 'EOS\n': #EOS: End Of Statement #EOSの部分を排除
            surface_list = line.split('\t')
            if len(surface_list) != 2 or surface_list[0] == '': #記号、空白などをスキップ
                continue
            else:
                tmp = surface_list[1].split(',')
                token = {'surface':surface_list[0], 'base':tmp[6], 'pos':tmp[0], 'pos1':tmp[1]}
                tokens.append(token)
        else:
            samples.append(tokens)
            tokens = [] #リセット

for num in samples[11]:
    print(num)

In [ ]:
# 参考https://qiita.com/kei_0324/items/400f639b2f185b39a0cf#pospart-of-speech%E3%82%BF%E3%82%B0%E3%81%A8%E3%81%AF

# 動詞
## 動詞の表層形をすべて抽出せよ．

In [ ]:
# ex) {'surface': 'つか', 'base': 'つく', 'pos': '動詞', 'pos1': '自立'}
verbs = set()
for sample in samples:
    for num in sample:
        if num['pos'] == '動詞':
            verbs.add(num['surface'])

for verb in list(verbs)[:30]:
    print(verb)

#  動詞の基本形
## 動詞の基本形をすべて抽出せよ．


In [ ]:
#ex) {'surface': 'つか', 'base': 'つく', 'pos': '動詞', 'pos1': '自立'}
#動詞で識別し、入力データは基本形に！
verb_bases = set()
for sample in samples:
    for num in sample:
        if num['pos'] == '動詞':
            verb_bases.add(num['base'])

for verb_base in list(verb_bases)[:30]:
    print(verb_base)

# AのB
## 2つの名詞が「の」で連結されている名詞句を抽出せよ

In [ ]:
# {'surface': '彼', 'base': '彼', 'pos': '名詞', 'pos1': '代名詞'}
# {'surface': 'の', 'base': 'の', 'pos': '助詞', 'pos1': '連体化'}
# {'surface': '掌', 'base': '掌', 'pos': '名詞', 'pos1': '一般'}
b_of_as = set()
for sample in samples:
    for num in range(1 , len(sample) - 1):
        if sample[num - 1]['pos'] == '名詞' and sample[num]['surface'] == 'の' and sample[num +1]['pos'] =='名詞':
            b_of_as.add(sample[num -1]['surface'] + sample[num]['surface'] + sample[num + 1]['surface'])

for b_of_a in list(b_of_as)[:30]:
    print(b_of_a)

# 名詞の連接
## 名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．

In [ ]:
# 最長一致https://www.megasoft.co.jp/mifes/seiki/about.html
noun_continue = set()
for sample in samples:
    nouns = ''
    num = 0
    for tmp in sample:
        if tmp['pos'] == '名詞':
            nouns = ''.join([nouns, tmp['surface']])
            num += 1
        elif num >= 2: #初期化
            noun_continue.add(nouns)
            nouns = ''
            num = 0
        else:
            nouns = ''
            num = 0
    if num >=2:
        noun_continue.add(nouns)

for i in list(noun_continue)[:30]:
    print(i)

# 単語の出現頻度
## 文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．

In [ ]:
from collections import defaultdict
word_freq = defaultdict(int)
for sample in samples:
    for tmp in sample:
        if tmp['pos'] != '記号':
            word_freq[tmp['base']] += 1

word_freq_sorted = sorted(word_freq.items(), key = lambda x:x[1], reverse=True)
word_freq_sorted

In [ ]:
# 参考https://analysis-navi.com/?p=2167
# https://techacademy.jp/magazine/19309

# 頻度上位10語
## 出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

In [ ]:
!pip install japanize_matplotlib #日本語対応

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

In [ ]:
x = [idx[0] for idx in word_freq_sorted[:10]] #key
y = [idx[1] for idx in word_freq_sorted[:10]] #volue
plt.figure(figsize=(20, 10))
plt.bar(x, y)
plt.show()

# 「猫」と共起頻度の高い上位10語
## 「猫」とよく共起する（共起頻度が高い）10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ

In [ ]:
word_freq_cat = defaultdict(int)
for sample in samples:
    if '猫' in [idx['surface'] for idx in sample]:
        for tmp in sample:
            if tmp['pos'] != '記号':
                word_freq_cat[tmp['base']] += 1
del word_freq_cat['猫']

#sort
word_freq_cat_soreted = sorted(word_freq_cat.items(), key=lambda x: x[1], reverse=True)

In [ ]:
x = [idx[0] for idx in word_freq_cat_soreted[:10]] #key
y = [idx[1] for idx in word_freq_cat_soreted[:10]] #volue
plt.figure(figsize=(20, 10))
plt.bar(x, y)
plt.show()

# ヒストグラム
## 単語の出現頻度のヒストグラムを描け．ただし，横軸は出現頻度を表し，1から単語の出現頻度の最大値までの線形目盛とする．縦軸はx軸で示される出現頻度となった単語の異なり数（種類数）である．

In [ ]:
values = word_freq.values()
plt.figure(figsize=(20, 10))
plt.hist(values, bins = 100)
plt.xlabel('出現頻度')
plt.ylabel('単語の種類数')
plt.show()

# Zipfの法則
## 単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．

In [ ]:
import numpy as np

x = [idx + 1 for idx in range(len(word_freq_sorted))]
y = [idx[1] for idx in word_freq_sorted]
plt.figure(figsize=(20, 10))
plt.scatter(np.log(x), np.log(y))
plt.show()

In [ ]:
# 参考https://controlabo.com/logplot/